In [2]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')



In [3]:
!pip install -q transformers

from transformers import pipeline

intent_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

intents = [
    "schedule appointment",
    "clinic hours",
    "insurance inquiry",
    "test results",
    "appointment cancellation",
    "prescription refill",
    "covid protocols",
    "update personal info",
    "services offered",
    "telehealth availability",
    "contact support"
]


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [5]:
faq_data = {
    "questions": [
        # Schedule appointment
        "How do I schedule an appointment?",
        "Can I book an appointment online?",
        "What is the process to make a doctor's appointment?",

        # Clinic hours
        "What are your clinic hours?",
        "Are you open on weekends?",
        "When does the clinic open and close?",

        # Insurance inquiry
        "Do you accept my insurance?",
        "Which insurance plans do you accept?",
        "Can I use my insurance for treatment?",

        # Test results
        "How can I get my test results?",
        "When will my lab results be ready?",
        "How do I access my blood test results?",

        # Appointment cancellation
        "How do I cancel or reschedule my appointment?",
        "Can I change my appointment time?",
        "What is the cancellation policy?",

        # Prescription refill
        "How do I refill my prescription?",
        "Can I get a refill for my medication online?",
        "What is the process to renew my prescription?",

        # COVID protocols
        "What COVID-19 protocols are in place?",
        "Do I need to wear a mask to my appointment?",
        "How is your clinic handling COVID-19 safety?",

        # Update personal info
        "How do I update my personal information?",
        "Can I change my address or phone number online?",
        "Where do I update my contact details?",

        # Services offered
        "What services do you offer?",
        "Do you provide pediatric care?",
        "What types of medical services are available?",

        # Telehealth availability
        "Are telehealth visits available?",
        "Can I book a virtual doctor visit?",
        "Do you offer video consultations?",

        # Contact support
        "How can I contact customer service?",
        "What is your support phone number?",
        "Who do I talk to for more help?"
    ],
    "answers": [
        # Schedule appointment answers
        "You can schedule an appointment by calling our clinic or using our online booking system.",
        "Yes, appointments can be booked online via our website or patient portal.",
        "To make a doctor's appointment, call our front desk or use the online scheduler.",

        # Clinic hours answers
        "Our clinic is open Monday to Friday from 8 AM to 6 PM, and Saturdays from 9 AM to 1 PM.",
        "Yes, we are open on Saturdays from 9 AM to 1 PM but closed on Sundays.",
        "We open at 8 AM and close at 6 PM on weekdays.",

        # Insurance inquiry answers
        "We accept most major insurance providers. Please contact us to verify your coverage.",
        "Our clinic accepts a variety of insurance plans including Aetna, Blue Cross, and United Healthcare.",
        "Yes, your insurance can be used for most treatments at our clinic.",

        # Test results answers
        "Test results are usually available within 3-5 business days and can be accessed through your patient portal.",
        "Lab results are typically ready within 3-5 days after testing.",
        "You can view your blood test results online through the patient portal.",

        # Appointment cancellation answers
        "To cancel or reschedule, please call us at least 24 hours before your appointment or use the patient portal.",
        "You can change your appointment time by contacting our front desk.",
        "Our cancellation policy requires 24-hour notice to avoid fees.",

        # Prescription refill answers
        "You can request prescription refills by contacting our pharmacy or through the patient portal.",
        "Refills can be requested online via your patient account.",
        "To renew your prescription, contact your doctor or use the online refill service.",

        # COVID protocols answers
        "We follow all recommended COVID-19 safety protocols, including mask mandates and social distancing.",
        "Masks are required for all visitors during appointments.",
        "Our clinic enforces strict COVID-19 guidelines to ensure patient safety.",

        # Update personal info answers
        "You can update your personal information by logging into your patient account or contacting our front desk.",
        "Changes to your address or phone number can be made online.",
        "Please update your contact details through the patient portal or by calling us.",

        # Services offered answers
        "Our services include primary care, pediatrics, lab testing, immunizations, and wellness checkups.",
        "Yes, we offer pediatric care as part of our services.",
        "We provide a wide range of medical services including preventive care and diagnostics.",

        # Telehealth availability answers
        "Telehealth visits are available for many of our services.",
        "You can book a virtual doctor visit online through our patient portal.",
        "Video consultations are offered for appropriate medical concerns.",

        # Contact support answers
        "You can contact customer service via phone at 1-800-123-4567 or email support@clinic.com.",
        "Our support phone number is 1-800-123-4567.",
        "For more help, please contact our support team by phone or email."
    ]
}

intent_tags = [
    "schedule appointment", "schedule appointment", "schedule appointment",
    "clinic hours", "clinic hours", "clinic hours",
    "insurance inquiry", "insurance inquiry", "insurance inquiry",
    "test results", "test results", "test results",
    "appointment cancellation", "appointment cancellation", "appointment cancellation",
    "prescription refill", "prescription refill", "prescription refill",
    "covid protocols", "covid protocols", "covid protocols",
    "update personal info", "update personal info", "update personal info",
    "services offered", "services offered", "services offered",
    "telehealth availability", "telehealth availability", "telehealth availability",
    "contact support", "contact support", "contact support"
]
faq_embeddings = model.encode(faq_data["questions"], convert_to_tensor=True)

In [7]:
import numpy as np

# Use 'model' which is the SentenceTransformer instance, not 'embedder'
faq_embeddings = model.encode(faq_data["questions"], convert_to_numpy=True)

faq_by_intent = {}
for intent in set(intent_tags):
    indices = [i for i, t in enumerate(intent_tags) if t == intent]
    faq_by_intent[intent] = {
        "questions": [faq_data["questions"][i] for i in indices],
        "answers": [faq_data["answers"][i] for i in indices],
        "embeddings": faq_embeddings[indices]
    }

def cosine_similarity(a, b):
    a_norm = a / np.linalg.norm(a)
    b_norm = b / np.linalg.norm(b)
    return np.dot(a_norm, b_norm)

In [8]:
def chatbot_response(user_question, threshold_intent=0.2, threshold_faq=0.2):
    # 1. Predict intent using zero-shot classification
    intent_result = intent_classifier(user_question, candidate_labels=list(set(intent_tags)))
    predicted_intent = intent_result['labels'][0]
    intent_confidence = intent_result['scores'][0]

    # 2. Get embedding for user question
    user_embedding = model.encode([user_question], convert_to_tensor=True)

    # 3. Calculate similarity with FAQs
    similarities = util.cos_sim(user_embedding, faq_embeddings)[0]
    top_indices = torch.topk(similarities, k=3).indices.tolist()

    # 4. Match top similar FAQs if any passes threshold
    for idx in top_indices:
        if similarities[idx] >= threshold_faq:
            return {
                "intent": predicted_intent,
                "intent_confidence": float(intent_confidence),
                "matched_question": faq_data["questions"][idx],
                "answer": faq_data["answers"][idx],
                "similarity": float(similarities[idx])
            }

    # 5. Fallback if no FAQ match is strong enough
    return {
        "intent": predicted_intent,
        "intent_confidence": float(intent_confidence),
        "matched_question": None,
        "answer": "I'm not sure about that. Please contact our support team for more help.",
        "similarity": None
    }


In [9]:
user_questions = [
    "Can I book an appointment online?",
    "What time do you close on Saturdays?",
    "How do I get my test results?",
    "Do you accept Blue Cross insurance?",
    "What are your COVID protocols?",
    "Can I get a refill for my medication?",
    "Who do I contact for help?"
]

for q in user_questions:
    response = chatbot_response(q)
    print(f"Q: {q}")
    print(f"A: {response['answer']}")
    print("-" * 60)


Q: Can I book an appointment online?
A: Yes, appointments can be booked online via our website or patient portal.
------------------------------------------------------------
Q: What time do you close on Saturdays?
A: Yes, we are open on Saturdays from 9 AM to 1 PM but closed on Sundays.
------------------------------------------------------------
Q: How do I get my test results?
A: Test results are usually available within 3-5 business days and can be accessed through your patient portal.
------------------------------------------------------------
Q: Do you accept Blue Cross insurance?
A: We accept most major insurance providers. Please contact us to verify your coverage.
------------------------------------------------------------
Q: What are your COVID protocols?
A: We follow all recommended COVID-19 safety protocols, including mask mandates and social distancing.
------------------------------------------------------------
Q: Can I get a refill for my medication?
A: Refills can be

In [10]:
!pip install gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.7 MB/s eta 0:00:00


In [11]:
import gradio as gr

# Define a wrapper for the chatbot to use in Gradio
def gradio_chat_interface(user_input):
    result = chatbot_response(user_input)
    return result['answer']

# Launch the Gradio interface
interface = gr.Interface(
    fn=gradio_chat_interface,
    inputs=gr.Textbox(lines=2, placeholder="Ask a health-related question..."),
    outputs="text",
    title="Healthcare Support Chatbot",
    description="Ask your questions related to appointments, test results, COVID protocols, telehealth, and more!"
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e04df9bd682bf9cec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
app_code = """
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import gradio as gr

# Load models
intent_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
model = SentenceTransformer('all-MiniLM-L6-v2')

# Intents list
intents = [
    "schedule appointment", "clinic hours", "insurance inquiry", "test results",
    "appointment cancellation", "prescription refill", "covid protocols",
    "update personal info", "services offered", "telehealth availability", "contact support"
]

# FAQ data and intents
faq_data = {
    "questions": [
        "How do I schedule an appointment?", "Can I book an appointment online?", "What is the process to make a doctor's appointment?",
        "What are your clinic hours?", "Are you open on weekends?", "When does the clinic open and close?",
        "Do you accept my insurance?", "Which insurance plans do you accept?", "Can I use my insurance for treatment?",
        "How can I get my test results?", "When will my lab results be ready?", "How do I access my blood test results?",
        "How do I cancel or reschedule my appointment?", "Can I change my appointment time?", "What is the cancellation policy?",
        "How do I refill my prescription?", "Can I get a refill for my medication online?", "What is the process to renew my prescription?",
        "What COVID-19 protocols are in place?", "Do I need to wear a mask to my appointment?", "How is your clinic handling COVID-19 safety?",
        "How do I update my personal information?", "Can I change my address or phone number online?", "Where do I update my contact details?",
        "What services do you offer?", "Do you provide pediatric care?", "What types of medical services are available?",
        "Are telehealth visits available?", "Can I book a virtual doctor visit?", "Do you offer video consultations?",
        "How can I contact customer service?", "What is your support phone number?", "Who do I talk to for more help?"
    ],
    "answers": [
        "You can schedule an appointment by calling our clinic or using our online booking system.",
        "Yes, appointments can be booked online via our website or patient portal.",
        "To make a doctor's appointment, call our front desk or use the online scheduler.",
        "Our clinic is open Monday to Friday from 8 AM to 6 PM, and Saturdays from 9 AM to 1 PM.",
        "Yes, we are open on Saturdays from 9 AM to 1 PM but closed on Sundays.",
        "We open at 8 AM and close at 6 PM on weekdays.",
        "We accept most major insurance providers. Please contact us to verify your coverage.",
        "Our clinic accepts a variety of insurance plans including Aetna, Blue Cross, and United Healthcare.",
        "Yes, your insurance can be used for most treatments at our clinic.",
        "Test results are usually available within 3-5 business days and can be accessed through your patient portal.",
        "Lab results are typically ready within 3-5 days after testing.",
        "You can view your blood test results online through the patient portal.",
        "To cancel or reschedule, please call us at least 24 hours before your appointment or use the patient portal.",
        "You can change your appointment time by contacting our front desk.",
        "Our cancellation policy requires 24-hour notice to avoid fees.",
        "You can request prescription refills by contacting our pharmacy or through the patient portal.",
        "Refills can be requested online via your patient account.",
        "To renew your prescription, contact your doctor or use the online refill service.",
        "We follow all recommended COVID-19 safety protocols, including mask mandates and social distancing.",
        "Masks are required for all visitors during appointments.",
        "Our clinic enforces strict COVID-19 guidelines to ensure patient safety.",
        "You can update your personal information by logging into your patient account or contacting our front desk.",
        "Changes to your address or phone number can be made online.",
        "Please update your contact details through the patient portal or by calling us.",
        "Our services include primary care, pediatrics, lab testing, immunizations, and wellness checkups.",
        "Yes, we offer pediatric care as part of our services.",
        "We provide a wide range of medical services including preventive care and diagnostics.",
        "Telehealth visits are available for many of our services.",
        "You can book a virtual doctor visit online through our patient portal.",
        "Video consultations are offered for appropriate medical concerns.",
        "You can contact customer service via phone at 1-800-123-4567 or email support@clinic.com.",
        "Our support phone number is 1-800-123-4567.",
        "For more help, please contact our support team by phone or email."
    ]
}

intent_tags = [
    "schedule appointment", "schedule appointment", "schedule appointment",
    "clinic hours", "clinic hours", "clinic hours",
    "insurance inquiry", "insurance inquiry", "insurance inquiry",
    "test results", "test results", "test results",
    "appointment cancellation", "appointment cancellation", "appointment cancellation",
    "prescription refill", "prescription refill", "prescription refill",
    "covid protocols", "covid protocols", "covid protocols",
    "update personal info", "update personal info", "update personal info",
    "services offered", "services offered", "services offered",
    "telehealth availability", "telehealth availability", "telehealth availability",
    "contact support", "contact support", "contact support"
]

def chatbot_response(user_question, threshold_intent=0.2, threshold_faq=0.4):
    user_embedding = model.encode([user_question], convert_to_tensor=True)

    # Intent prediction
    intent_result = intent_classifier(user_question, candidate_labels=intents)
    predicted_intent = intent_result['labels'][0]
    intent_confidence = intent_result['scores'][0]

    if intent_confidence < threshold_intent:
        return {"answer": "Sorry, I couldn't understand your question. Could you please rephrase?"}

    # Filter FAQ questions by predicted intent
    filtered_faqs = [
        (q, a) for q, a, intent_tag in zip(faq_data["questions"], faq_data["answers"], intent_tags)
        if intent_tag == predicted_intent
    ]
    if not filtered_faqs:
        return {"answer": "Sorry, I don't have an answer for that question yet."}

    # Encode filtered FAQ questions
    faq_embeddings = model.encode([q for q, _ in filtered_faqs], convert_to_tensor=True)

    # Calculate cosine similarities
    cosine_scores = (user_embedding @ faq_embeddings.T).cpu().numpy()[0]
    max_idx = np.argmax(cosine_scores)
    max_score = cosine_scores[max_idx]

    if max_score < threshold_faq:
        return {"answer": "Sorry, I couldn't find a good match for your question."}

    return {"answer": filtered_faqs[max_idx][1]}

# Gradio Interface
def respond(user_question):
    response = chatbot_response(user_question)
    return response["answer"]

iface = gr.Interface(fn=respond, inputs="text", outputs="text", title="Healthcare Support Chatbot",
                     description="Ask your questions related to appointments, test results, COVID protocols, telehealth, and more!")

if __name__ == "__main__":
    iface.launch()
"""

with open("app.py", "w") as f:
    f.write(app_code)

print("app.py created successfully!")


app.py created successfully!


In [13]:
from google.colab import files
files.download("app.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
requirements = """
transformers
sentence-transformers
gradio
torch
numpy
"""

with open("requirements.txt", "w") as f:
    f.write(requirements.strip())

print("requirements.txt created successfully!")


requirements.txt created successfully!


In [15]:
files.download("requirements.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>